In [ ]:
!pip install boto3

In [ ]:
import boto3
import json
import time

bedrock_region = "us-east-1"

bedrock_client = boto3.client('bedrock-runtime', region_name=bedrock_region)

In [ ]:
# Find out which CRIS profile to use
!/usr/local/bin/aws bedrock list-inference-profiles --region us-east-1

In [ ]:
# Mock ToolA: Returns a list of movies.
def toolA():
    return {
        "movies": [
            {"title": "Dune: Part Two", "year": 2024},
            {"title": "Blade Runner 2049", "year": 2017}
        ]
    }

# Mock ToolB: Returns revenue data given a list of movies.
def toolB(movies):
    revenue_data = {
        "Dune: Part Two": "$620M",
        "Blade Runner 2049": "$260M"
    }
    return {"revenues": {movie["title"]: revenue_data.get(movie["title"], "Unknown") for movie in movies}}

# Define tool data model to be passed to Claude.
tools = [
    {
        "name": "ToolA",
        "description": "Fetch a list of movies. Returns a JSON object with key 'movies' containing a list of movies with title and year.",
        "input_schema": {
            "type": "object",
            "properties": {},
            "required": []
        }
    },
    {
        "name": "ToolB",
        "description": "Fetch revenue data for given movies. Expects an object with a 'movies' key (an array of movie objects with 'title' and 'year').",
        "input_schema": {
            "type": "object",
            "properties": {
                "movies": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "title": {"type": "string"},
                            "year": {"type": "number"}
                        },
                        "required": ["title", "year"]
                    }
                }
            },
            "required": ["movies"]
        }
    }
]

In [ ]:
def invoke_claude(conversation, thinking=False, budget_tokens=2048):
    """
    Invokes Claude via Bedrock Runtime using a conversation array.
    The conversation is a list of message objects.
    """
    request_body = {
        "anthropic_version": "bedrock-2023-05-31",
        "messages": conversation,
        "max_tokens": 4096,            # Must be greater than thinking.budget_tokens
        "temperature": 1,              # Must be 1
        "stop_sequences": ["END"],     
        "tools": tools                 # Provide tool definitions
    }
    
    if thinking:
        request_body["thinking"] = {"type": "enabled", "budget_tokens": budget_tokens}
    
    # Inference Profile ARN for CIRS.
    inference_profile_arn = "arn:aws:bedrock:us-east-1:<YOUR_AWS_ACCOUNT>:inference-profile/us.anthropic.claude-3-7-sonnet-20250219-v1:0"
    
    start_time = time.time()
    response = bedrock_client.invoke_model(
        modelId=inference_profile_arn,
        contentType="application/json",
        accept="application/json",
        body=json.dumps(request_body)
    )
    elapsed_time = time.time() - start_time
    
    response_body = json.loads(response['body'].read().decode('utf-8'))
    
    output_text = response_body.get("completion", "")
    if not output_text:
        contents = response_body.get("content", [])
        output_text = " ".join(item.get("text", "") for item in contents if item.get("type") == "text")
    
    usage = response_body.get("usage", {})
    thinking_content = response_body.get("thinking", "")
    
    return output_text, usage, elapsed_time, thinking_content, response_body

In [ ]:
# Build the initial conversation
conversation = [
    {
        "role": "user",
        "content": (
            "You are an AI assistant with access to external tools. Your goal is to return a structured list of movies along with their revenues. "
            "If you have enough information, produce your final answer prefixed with 'FINAL ANSWER:'. Otherwise, if you need data, "
            "please use one of the provided tools (ToolA or ToolB) to gather the necessary data. "
            "When using a tool, output a tool_use block in your response."
        )
    }
]

iteration = 0
max_iterations = 5  # Prevent infinite loops

print("\n🔹 Starting AI Assistant Reasoning Process...\n")

while iteration < max_iterations:
    iteration += 1
    print(f"\n🔄 Iteration {iteration} - Sending conversation to Claude...\n")
    
    response_text, usage, time_taken, thinking_content, full_resp = invoke_claude(
        conversation=conversation,
        thinking=True,
        budget_tokens=2048
    )
    
    print("🧠 Claude's Response:\n", response_text)
    print("\n🔍 Thinking Content:\n", thinking_content)
    print("\n📊 Token Usage:", usage)
    print("⏱️ Execution Time:", time_taken, "seconds")
    
    # Append the assistant's response to the conversation
    assistant_message = {
        "role": "assistant",
        "content": full_resp.get("content", [])
    }
    conversation.append(assistant_message)
    
    # Check if Claude provided a final answer.
    if "FINAL ANSWER:" in response_text.upper():
        print("\n✅ Claude has provided the final answer.\n")
        break
    
    # Look for a tool_use block in the assistant's response.
    tool_use_block = None
    for block in assistant_message["content"]:
        if block.get("type") == "tool_use":
            tool_use_block = block
            break
    
    if tool_use_block is None:
        print("\n❌ No tool_use block found in Claude's response. Exiting loop...\n")
        break
    
    # Extract details from the tool_use block.
    tool_name = tool_use_block.get("name")
    tool_id = tool_use_block.get("id")
    tool_input = tool_use_block.get("input", {})
    
    print(f"\n📌 Detected tool use request: {tool_name} with id {tool_id} and input {tool_input}")
    
    # Run tool execution based on the tool name.
    if tool_name == "ToolA":
        tool_result_data = toolA()
    elif tool_name == "ToolB":
        # For ToolB, expect tool_input to contain a "movies" field.
        tool_result_data = toolB(tool_input.get("movies", []))
    else:
        print(f"\n❌ Unrecognized tool name: {tool_name}. Exiting loop...\n")
        break
    
    # Append a new user message with a tool_result block referencing the tool_use id
    tool_result_message = {
        "role": "user",
        "content": [
            {
                "type": "tool_result",
                "tool_use_id": tool_id,
                "content": json.dumps(tool_result_data)
            }
        ]
    }
    conversation.append(tool_result_message)
    
    # Print the conversation history for debugging.
    print("\n🔹 Current Conversation History:")
    for idx, msg in enumerate(conversation, 1):
        print(f"Message {idx}:", json.dumps(msg, indent=2))

print("\n🎉 Final AI Assistant Response:\n", response_text)


🔹 Starting AI Assistant Reasoning Process...


🔄 Iteration 1 - Sending conversation to Claude...

🧠 Claude's Response:
 I'll help you get a structured list of movies with their revenues. This will require two steps:

Step 1: First, I need to retrieve the list of movies.

🔍 Thinking Content:
 

📊 Token Usage: {'input_tokens': 629, 'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'output_tokens': 264}
⏱️ Execution Time: 12.051815748214722 seconds

📌 Detected tool use request: ToolA with id toolu_bdrk_01RZG7b2yZGQaggqe72rZgdk and input {}

🔹 Current Conversation History:
Message 1: {
  "role": "user",
  "content": "You are an AI assistant with access to external tools. Your goal is to return a structured list of movies along with their revenues. If you have enough information, produce your final answer prefixed with 'FINAL ANSWER:'. Otherwise, if you need data, please use one of the provided tools (ToolA or ToolB) to gather the necessary data. When using a tool, output a t